In [3]:
import numpy as np
import random
import nbimporter
import _TM as TM
import Encoder
import _SP3 as SP3

In [4]:
encoder = Encoder.ScalarEncoder(out_size=100, w=21, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [8]:
for epoch in range(3):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
#     print(a)
    for i in a:
        input_data = encoder.encode(i)        
        sp.compute_SDR(input_data)

[ 1  3 20 10  7 24 28  6 12 19  4 25 11 16 29 18  2  8  9 22 13  5 14 17 21
 27  0 23 26 15]
[10 26  4  3 17  6 29 12  0 19 24 15 21 13  7 16 28  2 23 22  8 18 11  1  9
 14 27 25  5 20]
[ 6 26  1  7 16 21  0  3 15 29 23 14 11  4  8 25 18 12 22 20 24 10  2  5 28
 27 17  9 13 19]


In [11]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)

유사도 : 0.7
[ 12  25 113 116 130 192 234 390 430 465]
[  3  12  25  47 113 130 192 234 383 430]
유사도 : 0.3
[  3  12  25  47 113 130 192 234 383 430]
[  4  12  25  78 119 122 187 221 234 350]
유사도 : 0.2
[  4  12  25  78 119 122 187 221 234 350]
[  4  60 117 122 144 198 199 364 365 479]
유사도 : 0.6
[  4  60 117 122 144 198 199 364 365 479]
[117 122 194 198 199 209 364 365 441 488]
유사도 : 0.6
[117 122 194 198 199 209 364 365 441 488]
[115 122 194 209 215 289 364 365 410 424 441]
유사도 : 0.5454545454545454
[115 122 194 209 215 289 364 365 410 424 441]
[ 38 122 181 194 213 215 289 364 404 410]
유사도 : 0.6
[ 38 122 181 194 213 215 289 364 404 410]
[115 122 169 213 215 289 320 364 410 441]
유사도 : 0.7
[115 122 169 213 215 289 320 364 410 441]
[122 169 176 213 215 240 289 375 410 441 448]
유사도 : 0.2727272727272727
[122 169 176 213 215 240 289 375 410 441 448]
[ 72  89 150 169 215 272 289 300 324 433]
유사도 : 0.6
[ 72  89 150 169 215 272 289 300 324 433]
[ 28  72  89 215 233 289 300 328 342 433]
유사도 : 0.4
[ 28

In [19]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, max_seg=5, threshold=7)

tm.learn = True

seq_data = [1,2,3,4,5]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()        
        tm.compute(sdr)

In [24]:
tm.learn = False
prev_pred = [ 3,  12,  25,  47 ,113 ,130 ,192 ,234, 383, 430,42]
for d in seq_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("Now columns (sdr) : {}".format(sp.getActiveColumns()))
    print("Predicted columns (예측Predict) : {}".format(tm.column_predicted()))
    print("Now burst : {}".format(tm.column_bursted()))
    print("\n")

예측 Accuracy : 1.0
Now columns (sdr) : [  3  12  25  47 113 130 192 234 383 430]
Predicted columns (예측Predict) : [4, 234, 12, 25, 187, 221, 350]
Now burst : [234, 383]


예측 Accuracy : 0.7
Now columns (sdr) : [  4  12  25  78 119 122 187 221 234 350]
Predicted columns (예측Predict) : [4, 198, 199, 364, 365, 144, 117, 122, 60, 479]
Now burst : [4, 12, 78, 119, 25, 122]


예측 Accuracy : 1.0
Now columns (sdr) : [  4  60 117 122 144 198 199 364 365 479]
Predicted columns (예측Predict) : [209, 122, 199]
Now burst : []


예측 Accuracy : 0.3
Now columns (sdr) : [117 122 194 198 199 209 364 365 441 488]
Predicted columns (예측Predict) : [289, 194, 424, 364, 365, 410, 209, 115, 215, 122]
Now burst : [194, 198, 488, 364, 365, 117, 441]


예측 Accuracy : 0.9090909090909091
Now columns (sdr) : [115 122 194 209 215 289 364 365 410 424 441]
Predicted columns (예측Predict) : [192, 130, 3, 234, 12, 430, 47, 113, 25, 383]
Now burst : [441]


